### 전처리 전 전체 지역 EDA

지역별 리뷰 데이터 수 

In [0]:
%sql 

-- 지역별 리뷰 개수 

select region, count(*) from hive_metastore.default.review_data
group by region; 

region,count(*)
Texas,66435184
Florida,61803524
California,70529977
New_York,33459761
Georgia,24060125
North_Carolina,22299136
Illinois,23096838
Ohio,23039365
Pennsylvania,21944802
Michigan,20776155


지역별 gmap_id 수 

In [0]:
%sql 
--  지역별 gmap_id 수 

SELECT 
    region, 
    COUNT(gmap_id) AS gmap_id_count
FROM 
    hive_metastore.default.meta_data
GROUP BY 
    region;

region,gmap_id_count
California,515961
Texas,447314
Florida,378020
New_York,272189
Illinois,179205
Pennsylvania,190816
Ohio,173761
North_Carolina,166235
Georgia,166381
Michigan,158819


지역별 user_id 수 

In [0]:
%sql 
--  지역별 user_id 수 
SELECT 
    region, 
    COUNT(DISTINCT user_id) AS distinct_user_count
FROM 
    hive_metastore.default.review_data
GROUP BY 
    region;

region,distinct_user_count
Maine,575257
New_Hampshire,748007
other,143319
Vermont,288211
South_Carolina,2964113
Arkansas,1217718
Oregon,2764082
Arizona,4020106
Idaho,968801
Nebraska,798181


In [0]:
%sql
-- user_id 가 null 값인 개수가 몇개인지 지역별로 보기 (중복 포함)
WITH total_user_count AS (
    SELECT region, COUNT(user_id) AS total_cnt
    FROM hive_metastore.default.review_data
    GROUP BY region
)
SELECT 
    rd.region, 
    COUNT(*) AS null_cnt, 
    t.total_cnt,
    COUNT(*) / t.total_cnt AS null_user_ratio
FROM 
    hive_metastore.default.review_data rd
JOIN 
    total_user_count t
ON 
    rd.region = t.region
WHERE 
    rd.user_id IS NULL
GROUP BY 
    rd.region, t.total_cnt
ORDER BY 
    null_cnt DESC;

In [0]:
%sql
--  User_id 기준 - 한 사용자가 몇 개의 gmap_id 장소를 갔는지 (user_id null 값 제외)
-- 장소 방문 개수별 분포 확인


select unique_gmap_count, count(*) as cnt
from 
  (select user_id, count(distinct gmap_id) as unique_gmap_count
  from hive_metastore.default.review_data
  where user_id is not null
  group by user_id)
group by unique_gmap_count
order by cnt desc;

지역별 gmap_id 기준 개수 분포 

In [0]:
%sql
-- gmap_id 기준 리뷰 개수 분포 확인 (unique 미적용시) 
-- 최소, 최대, 평균 

SELECT min(gmap_count), max(gmap_count), avg(gmap_count)
from 
  (SELECT gmap_id, COUNT(*) as gmap_count
  from hive_metastore.default.review_data
  group by gmap_id); 

In [0]:
%sql
-- 지역별 
--  gmap_id 기준 - 한 장소에 몇 명의 user_id가 방문(리뷰 작성)했는지
-- 리뷰 개수별 분포 확인

SELECT region, unique_user_count, COUNT(*) as cnt
FROM (
  SELECT gmap_id, region, COUNT(user_id) as user_count, COUNT(DISTINCT user_id) AS unique_user_count
  FROM hive_metastore.default.review_data
  WHERE user_id is not null
  GROUP BY gmap_id, region
) as subquery
GROUP BY region, unique_user_count
ORDER BY region, cnt DESC;

region - 사용자가 몇개 지역에 리뷰를 작성했는지

In [0]:
%sql
-- 사용자가 리뷰를 작성한 지역이 몇개 인지 
-- user_id 기준 지역 unique count
-- 분포

select region_count, count(*) as cnt
from 
  (select user_id, count(distinct region) as region_count
  from hive_metastore.default.review_data
  where user_id is not null
  group by user_id)
group by region_count
order by cnt desc;

### 실버 데이터 전체지역 EDA

year- 리뷰 작성년도 분포 

In [0]:
%sql
SELECT 
    YEAR(time) AS year, -- time 컬럼에서 년도를 추출하여 year 컬럼 생성
    COUNT(*) AS review_count -- 년도별 리뷰 개수를 카운트
FROM 
    hive_metastore.silver.review_data
GROUP BY 
    YEAR(time) -- year 컬럼을 기준으로 그룹화
ORDER BY 
    year; -- 년도별로 정렬

year,review_count
1990,899
1999,161
2000,20
2001,39
2002,320
2003,598
2004,1569
2005,2721
2006,5132
2007,8590


In [0]:
%sql
SELECT 
    region, -- 지역을 선택
    YEAR(time) AS year, -- time 컬럼에서 년도를 추출하여 year 컬럼 생성
    COUNT(*) AS review_count -- 년도별 리뷰 개수를 카운트
FROM 
    hive_metastore.silver.review_data
GROUP BY 
    region, YEAR(time) -- region과 year 컬럼을 기준으로 그룹화
ORDER BY 
    region, year; -- 지역과 년도별로 정렬

region,year,review_count
Alabama,1999,2
Alabama,2006,45
Alabama,2007,91
Alabama,2008,110
Alabama,2009,182
Alabama,2010,862
Alabama,2011,3576
Alabama,2012,8225
Alabama,2013,17159
Alabama,2014,25792


Databricks visualization. Run in Databricks to view.